In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import re

from collections import Counter
from ydata_profiling import ProfileReport

import transform_pd as tpd
import transform_pl as tpl

from utils import get_non_castable
from constants import *

In [2]:
%load_ext autoreload
%autoreload 2

# Pandas

## Read data

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [4]:
pd_df_train = pd.read_csv("train.csv")

/var/folders/rr/2my7n2l5511dqbvltxz0qyzh0000gq/T/ipykernel_90770/768096524.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_df_train = pd.read_csv("train.csv")


In [5]:
pd_df_train.columns[26]

'Monthly_Balance'

In [6]:
get_non_castable(pd_df_train, "Monthly_Balance", float)

['__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__',
 '__-333333333333333333333333333__']

In [7]:
NA_VALUES = {"Monthly_Balance": "__-333333333333333333333333333__"}
pd_df_train = pd.read_csv("train.csv", na_values=NA_VALUES)

In [8]:
pd_df_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.494089,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.629162,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.209863,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.451310,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.489231,Good


## Check validity

In [9]:
pd_df_train.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance             float64
Credit_Score                

In [10]:
for var in pd_df_train.columns:
    print(var)
    print("Examples of non-castable values: {}".format(get_non_castable(pd_df_train, var, float)[:10]))
    print("Unique values (potentially trimmed): {}".format(pd_df_train[var].unique()[:10]))
    print("")

ID
Examples of non-castable values: ['0x1602', '0x1603', '0x1604', '0x1605', '0x1606', '0x1607', '0x1608', '0x1609', '0x160e', '0x160f']
Unique values (potentially trimmed): ['0x1602' '0x1603' '0x1604' '0x1605' '0x1606' '0x1607' '0x1608' '0x1609'
 '0x160e' '0x160f']

Customer_ID
Examples of non-castable values: ['CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0xd40', 'CUS_0x21b1', 'CUS_0x21b1']
Unique values (potentially trimmed): ['CUS_0xd40' 'CUS_0x21b1' 'CUS_0x2dbc' 'CUS_0xb891' 'CUS_0x1cdb'
 'CUS_0x95ee' 'CUS_0x284a' 'CUS_0x5407' 'CUS_0x4157' 'CUS_0xba08']

Month
Examples of non-castable values: ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'January', 'February']
Unique values (potentially trimmed): ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August']

Name
Examples of non-castable values: ['Aaron Maashoh', 'Aaron Maashoh', 'Aaron Maashoh', 'Aaron Maashoh', 'Aaron Maashoh', 'Aaron Maashoh', 

### Type_of_Loan

In [11]:
split = pd_df_train.Type_of_Loan[~pd_df_train.Type_of_Loan.isna()].apply(lambda x: re.split(r", and |, ", x))
Counter([x for l in split.to_list() for x in l])

Counter({'Payday Loan': 40568,
         'Credit-Builder Loan': 40440,
         'Not Specified': 39616,
         'Home Equity Loan': 39104,
         'Student Loan': 38968,
         'Mortgage Loan': 38936,
         'Personal Loan': 38888,
         'Debt Consolidation Loan': 38776,
         'Auto Loan': 37992})

In [12]:
pd_df_train.head(30).to_csv("head.csv", index=False)

## Clean

### Replace with NaNs

In [13]:
pd_df_train_clean = tpd.cleanup_and_cast(pd_df_train)

### Initial EDA

In [14]:
pr = ProfileReport(pd_df_train_clean)

In [15]:
pr.to_file("eda.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Clean up outliers

In [16]:
pd_df_train_clean_no_outliers = tpd.replace_outliers(pd_df_train_clean)

In [17]:
for var in VARS_WITH_OUTLIERS_TOLERANCES.keys():
    print(var)
    desc = pd_df_train_clean_no_outliers[var].describe()
    print(desc)
    print("")

Age
count    97219.000000
mean        33.320009
std         10.769558
min         14.000000
25%         24.000000
50%         33.000000
75%         42.000000
max         56.000000
Name: Age, dtype: float64

Annual_Income
count     99001.000000
mean      50497.576053
std       38296.622971
min        7005.930000
25%       19344.270000
50%       36993.940000
75%       71676.000000
max      179987.280000
Name: Annual_Income, dtype: float64

Num_Bank_Accounts
count    98685.000000
mean         5.367624
std          2.593839
min         -1.000000
25%          3.000000
50%          5.000000
75%          7.000000
max         11.000000
Name: Num_Bank_Accounts, dtype: float64

Num_Credit_Card
count    97729.000000
mean         5.533322
std          2.067703
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max         11.000000
Name: Num_Credit_Card, dtype: float64

Interest_Rate
count    97966.000000
mean        14.531603
std          8.739748
min         

### EDA after removing outliers

In [18]:
pr_no_outliers = ProfileReport(pd_df_train_clean_no_outliers)
pr_no_outliers.to_file("eda_no_outliers.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Fill NaNs

In [19]:
pd_df_train_clean_no_outliers.apply(lambda x: x.isna().sum(), axis=0)

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                          2781
SSN                             0
Occupation                   7062
Annual_Income                 999
Monthly_Inhand_Salary       15002
Num_Bank_Accounts            1315
Num_Credit_Card              2271
Interest_Rate                2034
Num_of_Loan                  4348
Type_of_Loan                    0
Delay_from_due_date             0
Num_of_Delayed_Payment      28643
Changed_Credit_Limit         2091
Num_Credit_Inquiries        17774
Credit_Mix                  20195
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount       12007
Total_EMI_per_month          5404
Amount_invested_monthly      8784
Payment_Behaviour               0
Monthly_Balance              1209
Credit_Score                    0
dtype: int64

In [20]:
pd_df_train_clean_no_outliers_filled = tpd.fill_nas_from_other_customer_records(pd_df_train_clean_no_outliers)

In [21]:
pd_df_train_clean_no_outliers_filled.apply(lambda x: x.isna().sum(), axis=0)

ID                              0
Customer_ID                     0
Month                           0
Name                            0
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card              2271
Interest_Rate                2034
Num_of_Loan                  4348
Type_of_Loan                    0
Delay_from_due_date             0
Num_of_Delayed_Payment      28643
Changed_Credit_Limit         2091
Num_Credit_Inquiries        17774
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount       12007
Total_EMI_per_month          5404
Amount_invested_monthly      8784
Payment_Behaviour               0
Monthly_Balance              1209
Credit_Score                    0
dtype: int64

# Polars

## Read data

In [22]:
pl.Config.set_tbl_rows(100)

polars.config.Config

In [23]:
pl_df_train = pl.read_csv("train.csv")

ComputeError: Could not parse `__-333333333333333333333333333__` as dtype `f64` at column 'Monthly_Balance' (column number 27).
The current offset in the file is 12007648 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `__-333333333333333333333333333__` to the `null_values` list.

In [24]:
pl_df_train = pl.read_csv("train.csv", dtypes={"Monthly_Balance": pl.Utf8})

## Check validity

In [25]:
pl_df_train.dtypes

[Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Float64,
 Int64,
 Int64,
 Int64,
 Utf8,
 Utf8,
 Int64,
 Utf8,
 Utf8,
 Float64,
 Utf8,
 Utf8,
 Float64,
 Utf8,
 Utf8,
 Float64,
 Utf8,
 Utf8,
 Utf8,
 Utf8]

In [26]:
pl_df_train.schema

OrderedDict([('ID', Utf8),
             ('Customer_ID', Utf8),
             ('Month', Utf8),
             ('Name', Utf8),
             ('Age', Utf8),
             ('SSN', Utf8),
             ('Occupation', Utf8),
             ('Annual_Income', Utf8),
             ('Monthly_Inhand_Salary', Float64),
             ('Num_Bank_Accounts', Int64),
             ('Num_Credit_Card', Int64),
             ('Interest_Rate', Int64),
             ('Num_of_Loan', Utf8),
             ('Type_of_Loan', Utf8),
             ('Delay_from_due_date', Int64),
             ('Num_of_Delayed_Payment', Utf8),
             ('Changed_Credit_Limit', Utf8),
             ('Num_Credit_Inquiries', Float64),
             ('Credit_Mix', Utf8),
             ('Outstanding_Debt', Utf8),
             ('Credit_Utilization_Ratio', Float64),
             ('Credit_History_Age', Utf8),
             ('Payment_of_Min_Amount', Utf8),
             ('Total_EMI_per_month', Float64),
             ('Amount_invested_monthly', Utf8),
       

### Type_of_Loan

In [27]:
split = (
    pl_df_train["Type_of_Loan"]
    .drop_nulls()
    .map_elements(lambda x: re.split(r", and |, ", x))
)
Counter([x for l in split.to_list() for x in l])

Counter({'Payday Loan': 40568,
         'Credit-Builder Loan': 40440,
         'Not Specified': 39616,
         'Home Equity Loan': 39104,
         'Student Loan': 38968,
         'Mortgage Loan': 38936,
         'Personal Loan': 38888,
         'Debt Consolidation Loan': 38776,
         'Auto Loan': 37992})

## Clean

### Replace with NaNs

In [28]:
pl_df_train_clean = tpl.cleanup_and_cast(pl_df_train)

### Clean up outliers

In [29]:
pl_df_train_clean_no_outliers = tpl.replace_outliers(pl_df_train_clean)

In [30]:
for var in VARS_WITH_OUTLIERS_TOLERANCES.keys():
    print(var)
    desc = pl_df_train_clean_no_outliers[var].describe()
    print(desc)
    print("")

Age
shape: (9, 2)
┌────────────┬───────────┐
│ statistic  ┆ value     │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 100000.0  │
│ null_count ┆ 2781.0    │
│ mean       ┆ 33.320009 │
│ std        ┆ 10.769558 │
│ min        ┆ 14.0      │
│ 25%        ┆ 24.0      │
│ 50%        ┆ 33.0      │
│ 75%        ┆ 42.0      │
│ max        ┆ 56.0      │
└────────────┴───────────┘

Annual_Income
shape: (9, 2)
┌────────────┬──────────────┐
│ statistic  ┆ value        │
│ ---        ┆ ---          │
│ str        ┆ f64          │
╞════════════╪══════════════╡
│ count      ┆ 100000.0     │
│ null_count ┆ 999.0        │
│ mean       ┆ 50497.576053 │
│ std        ┆ 38296.622971 │
│ min        ┆ 7005.93      │
│ 25%        ┆ 19344.27     │
│ 50%        ┆ 36993.94     │
│ 75%        ┆ 71676.0      │
│ max        ┆ 179987.28    │
└────────────┴──────────────┘

Num_Bank_Accounts
shape: (9, 2)
┌────────────┬──────────┐
│ statistic  ┆ value    │
│ ---        

### Fill NaNs

In [31]:
pl_df_train_clean_no_outliers.select(pl.all().is_null().sum()).melt()

variable,value
str,u32
"""ID""",0
"""Customer_ID""",0
"""Month""",0
"""Name""",9985
"""Age""",2781
"""SSN""",0
"""Occupation""",7062
"""Annual_Income""",999
"""Monthly_Inhand…",15002


In [32]:
pl_df_train_clean_no_outliers_filled = tpl.fill_nas_from_other_customer_records(pl_df_train_clean_no_outliers)

In [33]:
pl_df_train_clean_no_outliers_filled.select(pl.all().is_null().sum()).melt()

variable,value
str,u32
"""ID""",0
"""Customer_ID""",0
"""Month""",0
"""Name""",0
"""Age""",0
"""SSN""",0
"""Occupation""",0
"""Annual_Income""",0
"""Monthly_Inhand…",0
